In [1]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
from icecream import ic, install
import numpy as np
import pandas as pd

import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2

# from itables import init_notebook_mode

# init_notebook_mode(
#     all_interactive=True,
#     connected=True,
# )

In [2]:
conn = connect("../../django/TargetDB/db.sqlite3")

session = tlc.ObservingSession(ap.Observer.at_site("APO"))
session.add_full_day("2025-03-15")
session.add_full_day("2025-03-16")
session.add_half_day("2025-03-18", first_half=False)
session.add_full_day("2025-03-19")

phase_event_defs = [
    ph.PhaseEventDef("Not in Eclipse", partial(ph.calc_time_of_gress, ingress=False)),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, ingress=True)),
]

min_altitude = 40 * u.deg

name = "APO Observing List 2025-03-15 to -19"
creator = tlc.TargetListCreator(name=name, connection=conn)
creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    partial(
        tlc.add_observability,
        observing_session=session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    # partial(tlc.filter_targets, criteria=lambda df: (df["Vmag"] < 11)),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=session, phase_event_defs=phase_event_defs),
    tlc.add_speckle,
    partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
]
tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, "html")
print(tl.summarize())


APO Observing List 2025-03-15 to -19
91 targets:
      91 QuadEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Observable: (5, 16)
    Count: (1, 0)
Associated tables:
       4 rows,  2 columns: Lunar Phases
     300 rows,  6 columns: Ephem
    1582 rows, 10 columns: Phase Events
     105 rows,  6 columns: Speckle
     351 rows,  7 columns: Speckle Phases



In [6]:
altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
# pr.calculate_eclipse_priority(pl, no_eclipse_score=0, min_altitude=30 * u.deg)
# pr.prioritize_phase_sequence(pl, ["Not in Eclipse", "Eclipse", "Not in Eclipse"], "Full Eclipse", True, True, True)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
pr.prioritize_side_observation(pl, side_phases=["Eclipse"])
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.1)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, "html")
# pl.categorical_priorities[0]

In [4]:
q2_session = tlc.ObservingSession(ap.Observer.at_site("APO"))
q2_session.add_day_range("2025-05-07", "2025-06-10")

creator = tlc.TargetListCreator(name="APO 2025-Q2 Proposal", connection=conn)

creator.steps = [
    tlc.add_targets,
    partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
    partial(
        tlc.filter_targets, inverse=True, criteria=lambda df: (df["Target Name"].isin(pl.overall_category_members["* * *"]))
    ),
    partial(tlc.filter_targets, inverse=True, criteria=lambda df: (df["Target Name"].isin(pl.overall_category_members["* *"]))),
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="TESS "),
    partial(
        tlc.add_observability,
        observing_session=q2_session,
        calc_moon_distance=True,
        observability_threshold=(min_altitude, 80 * u.deg),
    ),
    partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
    # partial(tlc.filter_targets, criteria=lambda df: (df["Vmag"] < 11)),
    tlc.add_ephemerides,
    partial(tlc.add_phase_events, observing_session=q2_session, phase_event_defs=phase_event_defs),
    tlc.add_speckle,
    partial(tlc.add_speckle_phase, phase_event_defs=phase_event_defs),
]

q2_tl = creator.calculate()
print(q2_tl.summarize())

C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:402: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list[max_alt_column] = segment_max_alts
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:413: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  answer.target_list[moon_dist_column] = dist
C:\Users\User\Dropbox\Astro\astropaul\astropaul\targetlistcreator.py:390: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whic

APO 2025-Q2 Proposal
49 targets:
      49 QuadEB
Column Count (primary, secondary):
    Target: (1, 2)
    Coordinates: (6, 3)
    Observable: (5, 140)
    Count: (1, 0)
Associated tables:
      35 rows,  2 columns: Lunar Phases
     157 rows,  6 columns: Ephem
    7015 rows, 10 columns: Phase Events
      72 rows,  6 columns: Speckle
     232 rows,  7 columns: Speckle Phases



In [7]:
q2_pl = pr.PriorityList(q2_tl, q2_session, interval=30 * u.min)
pr.calculate_altitude_priority(q2_pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(q2_pl, ["Eclipse"], "Eclipse", True, True, True)
pr.prioritize_side_observation(q2_pl, side_phases=["Eclipse"])
pr.calculate_overall_priority(q2_pl)
pr.aggregate_target_priorities(q2_pl, skip_column_threshold=0.1)
q2_pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(q2_tl, q2_pl, {}, "q2")
# pl.categorical_priorities[0]